<a href="https://colab.research.google.com/github/jaehyung-web/toilet-123/blob/main/%ED%99%94%EC%9E%A5%EC%8B%A4_%EC%BD%94%EB%93%9C_%EC%A4%91%EA%B0%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# app.py
# Flask 서버 + Static 이미지 지원 (Colab 구글 드라이브 마운트 포함)
import subprocess, sys
# 런타임에 반드시 필요한 패키지 설치
subprocess.check_call([sys.executable, "-m", "pip", "install", "flask", "pyngrok"] , stdout=subprocess.DEVNULL)

from flask import Flask, request, render_template_string, send_from_directory
import os, shutil

# Colab 환경에서 구글 드라이브 자동 마운트 (이미 마운트된 경우 건너뜀)
try:
    from google.colab import drive
    if not os.path.exists('/content/drive/MyDrive'):
        drive.mount('/content/drive')
except ImportError:
    pass

# ngrok 필수 사용
from pyngrok import ngrok
ngrok.set_auth_token("2zdNYkwML41M7L22D8Cd8wdW5xZ_74XsAp7x9DtveZ7xnHdgn")

app = Flask(__name__)

# 사용자 설정: 구글 드라이브 내 이미지 경로 (Colab 전용)
LOGO_DRIVE_PATH = "/content/drive/MyDrive/static/images (5).png"  # 실제 경로로 수정하세요

# static 폴더 준비 및 이미지 복사
os.makedirs('static', exist_ok=True)
if os.path.exists(LOGO_DRIVE_PATH):
    shutil.copy(LOGO_DRIVE_PATH, 'static/logo.png')
elif not os.path.exists('static/logo.png'):
    open('static/logo.png','a').close()

# 화장실 상태 데이터
stall_status = {
    "본관": {"2":{"좌측":{str(i):0 for i in range(1,5)},"우측":{str(i):0 for i in range(1,5)}},
             "3":{"좌측":{str(i):0 for i in range(1,5)},"우측":{str(i):0 for i in range(1,5)}},
             "4":{"좌측":{str(i):0 for i in range(1,5)},"우측":{str(i):0 for i in range(1,5)}},
             "5":{"좌측":{str(i):0 for i in range(1,5)},"우측":{str(i):0 for i in range(1,5)}}},
    "아단관": {"4":{"큰 화장실":{str(i):0 for i in range(1,8)},"작은 화장실":{str(i):0 for i in range(1,4)}},
              "5":{"큰 화장실":{str(i):0 for i in range(1,8)},"작은 화장실":{str(i):0 for i in range(1,4)}}},
    "창의관": {str(f):{"화장실":{str(i):0 for i in range(1,5)}} for f in range(1,6)},
    "현암관":{"B1":{"화장실":{str(i):0 for i in range(1,6)}},"1":{"화장실":{str(i):0 for i in range(1,6)}}},
    "여고":{str(f):{"화장실":{str(i):0 for i in range(1,5)}} for f in range(2,6)},
    "여송학사":{str(f):{"A동":{str(i):0 for i in range(1,5)},"B동":{str(i):0 for i in range(1,5)}} for f in range(1,5)},
    "야구장":{}
}

# 센서 업데이트 라우트
@app.route('/update')
def update():
    b = request.args.get('building')
    f = request.args.get('floor')
    loc = request.args.get('location')
    sid = request.args.get('stall')
    occ = request.args.get('occupied')
    if all([b in stall_status,
            f in stall_status[b],
            loc in stall_status[b][f],
            sid in stall_status[b][f][loc],
            occ in ['0','1']]):
        stall_status[b][f][loc][sid] = int(occ)
        return 'OK',200
    return 'Bad Request',400

# static 파일 서빙
@app.route('/static/<path:filename>')
def static_files(filename):
    return send_from_directory('static', filename)

# 메인 페이지 템플릿
main_page = '''
<!doctype html>
<html lang='ko'>
<head><meta charset='utf-8'>
<meta name='viewport' content='width=device-width,initial-scale=1.0'>
<title>BUGIL 실시간 화장실 정보 시스템</title>
<style>
  body{margin:0;padding:0;font-family:Arial,sans-serif;text-align:center;}
  h1{margin:20px 0;font-size:28px;}
  h2{margin:10px 0 30px;font-size:24px;color:#555;}
  .grid{display:grid;grid-template-columns:repeat(auto-fit,minmax(160px,1fr));gap:16px;padding:16px;}
  button{width:100%;height:120px;font-size:22px;font-weight:bold;border:none;border-radius:12px;background:#f39733;color:#fff;cursor:pointer;transition:0.2s;}
  button:hover{background:#d37f2e;}
  /* 로고 버튼만 배경 제거 */
  button.logo-btn { background: none; }
  button.logo-btn:hover { background: none; }
  img.logo{width:100%;height:100%;object-fit:contain;}
</style>
</head>
<body>
<h1>BUGIL 실시간 화장실 정보 시스템</h1>
<h2>화장실을 선택하십시오.</h2>
<div class='grid'>
  {% for b in buildings %}
  <form action='/building/{{ b }}'><button>{{ b }}</button></form>
  {% endfor %}
  <!-- 이미지 버튼 -->
  <form action='#'><button type='button' class='logo-btn'><img src='/static/logo.png' class='logo'></button></form>
</div>
</body>
</html>
'''

# 건물 페이지 템플릿
building_page = '''
<!doctype html>
<html lang='ko'>
<head><meta charset='utf-8'>
<meta name='viewport' content='width=device-width,initial-scale=1.0'>
<title>{{ building }} 현황</title>
<style>
  body{margin:0;padding:16px;font-family:Arial,sans-serif;}
  h1{font-size:24px;margin-bottom:12px;}
  h2{font-size:20px;margin:16px 0 8px;}
  h3{font-size:18px;margin:12px 0 6px;}
  .container{display:grid;grid-template-columns:repeat(auto-fit,minmax(100px,1fr));gap:8px;margin-bottom:12px;}
  .box{padding:16px;text-align:center;border-radius:8px;color:#fff;font-weight:bold;font-size:16px;}
  .occupied{background:#e74c3c;}
  .vacant{background:#2ecc71;}
  a.back{display:inline-block;margin-top:16px;padding:12px 24px;font-size:18px;background:#555;color:#fff;border-radius:8px;text-decoration:none;}
  a.back:hover{background:#333;}
</style>
</head>
<body>
<h1>{{ building }} 화장실 현황</h1>
{% for floor, places in data.items() %}
  <h2>{{ floor }}층</h2>
  {% for loc, stalls in places.items() %}
    <h3>{{ loc }}</h3>
    <div class='container'>
      {% for sid, occ in stalls.items() %}
        <div class='box {{ 'occupied' if occ else 'vacant' }}'>{{ sid }}번<br>{{ '사용중' if occ else '비어있음' }}</div>
      {% endfor %}
    </div>
  {% endfor %}
{% endfor %}
<a href='/' class='back'>⬅ 돌아가기</a>
</body>
</html>
'''

@app.route('/')
def home():
    return render_template_string(main_page, buildings=stall_status.keys())

@app.route('/building/<b>')
def show_building(b):
    if b in stall_status:
        return render_template_string(building_page, building=b, data=stall_status[b])
    return 'Not Found',404

# ngrok 터널링
url = ngrok.connect(5000)
print('✅ ngrok URL:', url)

if __name__=='__main__':
    app.run(port=5000)

ERROR:root:Unexpected exception finding object shape
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/google/colab/_debugpy_repr.py", line 54, in get_shape
    shape = getattr(obj, 'shape', None)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/werkzeug/local.py", line 318, in __get__
    obj = instance._get_current_object()
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/werkzeug/local.py", line 519, in _get_current_object
    raise RuntimeError(unbound_message) from None
RuntimeError: Working outside of request context.

This typically means that you attempted to use functionality that needed
an active HTTP request. Consult the documentation on testing for
information about how to avoid this problem.


ERROR:root:Unexpected exception finding object shape
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/google/colab/_debugpy_repr.py", line 54, in get_shape
    shape = getattr(obj, 'shape', None)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/werkzeug/local.py", line 318, in __get__
    obj = instance._get_current_object()
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/werkzeug/local.py", line 519, in _get_current_object
    raise RuntimeError(unbound_message) from None
RuntimeError: Working outside of request context.

This typically means that you attempted to use functionality that needed
an active HTTP request. Consult the documentation on testing for
information about how to avoid this problem.


ERROR:root:Unexpected exception finding object shape
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/google/colab/_debugpy_repr.py", line 54, in get_shape
    shape = getattr(obj, 'shape', None)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/werkzeug/local.py", line 318, in __get__
    obj = instance._get_current_object()
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/werkzeug/local.py", line 519, in _get_current_object
    raise RuntimeError(unbound_message) from None
RuntimeError: Working outside of request context.

This typically means that you attempted to use functionality that needed
an active HTTP request. Consult the documentation on testing for
information about how to avoid this problem.
ERROR:root:Unexpected exception finding object shape
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/google/colab/_debugpy_repr.py", line 5

✅ ngrok URL: NgrokTunnel: "https://2b4b524587cc.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
ERROR:root:Unexpected exception finding object shape
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/google/colab/_debugpy_repr.py", line 54, in get_shape
    shape = getattr(obj, 'shape', None)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/werkzeug/local.py", line 318, in __get__
    obj = instance._get_current_object()
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/werkzeug/local.py", line 519, in _get_current_object
    raise RuntimeError(unbound_message) from None
RuntimeError: Working outside of request context.

This typically means that you attempted to use functionality that needed
an active HTTP request. Consult the documentation on testing for
information abo